In [1]:
import SALib
from SALib.sample import saltelli
from SALib.analyze import sobol
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
#from tqdm import tqdm  # For progress bar
from ABM_Final_Final import SocialNetwork

In [2]:
# We define our variables and bounds
problem = {
    'num_vars': 5,
    'names': ['prob', 'mu', 'temp', 'conf_mu', 'tol_mu'],
    'bounds': [[0., 1.], [0.1,1], [0.1,1], [0., 1.], [0.,1.]]
}

# Set the repetitions, the amount of steps, and the amount of distinct values per variable
replicates = 1
max_steps = 800
distinct_samples = 4

# We get all our samples here
param_values = saltelli.sample(problem, distinct_samples)
    

# Initialize DataFrame to store results
num_samples = len(param_values) * replicates

data = pd.DataFrame(index=range(num_samples), columns=['prob', 'w_pop', 'w_prox', 'w_sim', 'mu', 'temp', 'conf_mu', 'tol_mu'])
print(data)
data['Run'], data['max IN degree'], data['avg clustering coeff'] = None, None, None

   prob w_pop w_prox w_sim   mu temp conf_mu tol_mu
0   NaN   NaN    NaN   NaN  NaN  NaN     NaN    NaN
1   NaN   NaN    NaN   NaN  NaN  NaN     NaN    NaN
2   NaN   NaN    NaN   NaN  NaN  NaN     NaN    NaN
3   NaN   NaN    NaN   NaN  NaN  NaN     NaN    NaN
4   NaN   NaN    NaN   NaN  NaN  NaN     NaN    NaN
5   NaN   NaN    NaN   NaN  NaN  NaN     NaN    NaN
6   NaN   NaN    NaN   NaN  NaN  NaN     NaN    NaN
7   NaN   NaN    NaN   NaN  NaN  NaN     NaN    NaN
8   NaN   NaN    NaN   NaN  NaN  NaN     NaN    NaN
9   NaN   NaN    NaN   NaN  NaN  NaN     NaN    NaN
10  NaN   NaN    NaN   NaN  NaN  NaN     NaN    NaN
11  NaN   NaN    NaN   NaN  NaN  NaN     NaN    NaN
12  NaN   NaN    NaN   NaN  NaN  NaN     NaN    NaN
13  NaN   NaN    NaN   NaN  NaN  NaN     NaN    NaN
14  NaN   NaN    NaN   NaN  NaN  NaN     NaN    NaN
15  NaN   NaN    NaN   NaN  NaN  NaN     NaN    NaN
16  NaN   NaN    NaN   NaN  NaN  NaN     NaN    NaN
17  NaN   NaN    NaN   NaN  NaN  NaN     NaN    NaN
18  NaN   Na

C:\Users\franc\AppData\Local\Temp\ipykernel_17468\2213441400.py:14: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5.1 Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, distinct_samples)


In [3]:
# Define your model
def run_model(params,max_steps):
    n_agents = 100
    prob, temp, mu, conf_mu, tol_mu = params
    w_sim = 0.2
    w_pop = 0.4
    w_prox = 0.2
    model = SocialNetwork(n_agents, prob, mu, w_pop, w_prox, w_sim, temp, conf_mu, tol_mu)
    results = pd.DataFrame(model.Data_Collector)
    for k in range(max_steps + 1):
        model.step()
        print(f"\rProgress: {(k / max_steps) * 100:.2f}%", end='', flush=True)
   
    df_results = pd.DataFrame(model.Data_Collector)
    
    return df_results['avg clustering coeff'].iloc[-1], df_results['max IN degrees'].iloc[-1]

In [5]:
n_jobs = 6

Y = Parallel(n_jobs=n_jobs)(delayed(run_model)(vals, max_steps) for vals in param_values)
print(Y)

In [26]:
# Batch run
n_jobs = -1
count = 1
for i in range(replicates):
    for vals in param_values:
        # Run the model with the current set of parameters
        print(f"\nRun {count}")
        clustering_coeff, max_in = Parallel(n_jobs=n_jobs)(delayed(run_model(vals, max_steps)))
        data.iloc[count, 0:len(vals)] = list(vals)
        data.iloc[count, len(vals):len(vals)+3] = [count, clustering_coeff, max_in]
        count += 1
        
data.to_csv(f"sobol_rep{replicates}_maxstep{max_steps}_distsamples{distinct_samples}.csv", index=False)



Run 1
Progress: 100.00%

TypeError: 'function' object is not iterable

In [ ]:
Si_clustering = sobol.analyze(problem, data['avg clustering coeff'].values, print_to_console=True)
Si_IN_Degree = sobol.analyze(problem, data['max IN degrees'].values, print_to_console=True)